# 2. component_extraction_CRF
1. To extract features and labels, sentences without quotations are not removed.
2. CRF
3. Evaluation

In [5]:
import pandas as pd 
import csv
import sklearn
from sklearn import metrics
from sklearn.metrics import classification_report

1. To extract features and labels

In [7]:
# to prepare for using CRF.py, not removing unlabeled sentences
preprocessed_data=["polnear_preprocessed_BIO_train.csv", "polnear_preprocessed_BIO_dev.csv",
                   "parc3_preprocessed_BIO_train.csv", "parc3_preprocessed_BIO_dev.csv",
                   "merged-parc3-polnear_preprocessed_BIO_dev.csv", "merged-parc3-polnear_preprocessed_BIO_train.csv"]
for i in preprocessed_data:
    with open(f"../Preprocessed_data/{i}", "r") as source:
        reader = csv.reader(source)
        with open(f"Preprocessed_data/{i.replace('.csv','_token_label.csv')}", "w") as result:
            writer = csv.writer(result, delimiter='\t')
            for r in reader:
                writer.writerow((r[1], r[2], r[3], r[4], r[5], r[6], r[7], r[8], r[9], r[10], r[11]))
            # to add an empty row at the end, otherwise CRF doesn't catch the last line. Because CRF is trained on sentences, and sentences are separated by ''.
            footers=['', '', '', '', '', '', '', '', '', '', 'O']
            footer_row='\t'.join(footers)
            writer.writerow(footer_row)
    file1 = open(f"../Preprocessed_data/{i.replace('.csv','_token_label.csv')}")
    reader1 = csv.reader(file1)
    lines= len(list(reader1))

    print(lines)

756666
81574
1109421
34145
115717
1866085


In [ ]:
# # to extract extra features, to prepare for using CRF.py, not removing unlabeled sentences
# # to extract only token and labels, to prepare for using CRF.py, removing unlabeled sentences
# preprocessed_data_token_label=["polnear_preprocessed_BIO_train_token_label.csv",
#                                "parc3_preprocessed_BIO_train_token_label.csv",
#                                "merged-parc3-polnear_preprocessed_BIO_train_token_label.csv"
#                               ]

# def get_extra_features(filename):
#     quote_opening = "``"
#     quote_closing = "''"

#     # Read file
#     csvinput = open(f"../Preprocessed_data/{filename}",'r')
#     csvreader = csv.reader(csvinput,delimiter='\t')
# #     print(csvreader)
#     headers = next(csvreader)
#     sents = []
#     current_sent = []
#     for row in csvreader:
#         current_sent.append(list(row))
#         #note that this is a simplification that works well for this particular data, in other situations, you may need to do more advanced preprocessing to identify sentence boundaries
#         if row[6] == ".":
#             sents.append(current_sent)
#             current_sent = []            

#     # Loop over sentences in file
#     in_quote = False
#     sents_extra_features = []
#     for sent in sents:
#         print(type(sent))
#         # Loop over tokens in sentence
#         sent_extra_features = sent.copy()
#         for token in sent:
#             token_extra_features = token.copy()
#             label = token[-1]
#             sent_extra_features.append(token_extra_features)
        
#         # Append row with extra features to new list
#         sents_extra_features.append(sent_extra_features)
            
#     # Write file
#     with open(f"../Preprocessed_data/{filename.replace('.csv','_extra_features.csv')}", "w") as result:
#         headers1 = ["Article_Name", "Sentence_nr", "Nr_in_file", "Nr_in_sentence", "FromTO", "Word", "Lemma", "POS", "Dep_label", "Token_dep_head", 
#                     "AR_label"]  # , "In_quote", ]
#         writer = csv.writer(result, delimiter='\t')
#         writer.writerow(headers1)
#         for f in sents_removed:
#             writer.writerows(f)
#     return

# # For every filename, get extra features
# for filename in preprocessed_data_token_label:
#     get_extra_features(filename)

In [19]:
# to extract only token and labels, to prepare for using CRF.py, removing unlabeled sentences
preprocessed_data_token_label=["polnear_preprocessed_BIO_train_token_label.csv",
                               "parc3_preprocessed_BIO_train_token_label.csv",
                               "merged-parc3-polnear_preprocessed_BIO_train_token_label.csv"
                              ]
#inputfile="polnear_preprocessed_BIO_dev_token_label.csv"
#inputfile=polnear_outfile_path
#for i in preprocessed_data_token_label:
#i="polnear_preprocessed_BIO_dev_token_label.csv"
def get_removed_file(filename):
#     print('something')
    csvinput = open(f"../Preprocessed_data/{filename}",'r')
    csvreader = csv.reader(csvinput,delimiter='\t')
#     print(csvreader)
    headers=next(csvreader)
    sents = []
    current_sent = []
    for row in csvreader:
        current_sent.append(list(row))
        #note that this is a simplification that works well for this particular data, in other situations, you may need to do more advanced preprocessing to identify sentence boundaries
        if row[6] == ".":
            sents.append(current_sent)
            current_sent = []
    sents_removed=[]
    for sent in sents:
#         print(sents)
        sent_label_list=[]
        for f in sent:
            label = f[-1]
            sent_label_list.append(label)
        if set(sent_label_list)!={'O'}:
            #print(set(sent))
            sents_removed.append(sent)
    with open(f"../Preprocessed_data/{filename.replace('.csv','_removed.csv')}", "w") as result:
#         headers1=['Word','AR_label']
        headers1 = ["Article_Name", "Sentence_nr", "Nr_in_file", "Nr_in_sentence", "FromTo", "Word", "Lemma", "POS", "Dep_label", "Token_dep_head", "AR_label"]
        #header_row=','.join(headers)+'\n'    
        writer = csv.writer(result, delimiter='\t')
        writer.writerow(headers1)
        for f in sents_removed:
            writer.writerows(f)
    return

for i in preprocessed_data_token_label:
    get_removed_file(i)

2. Run CRF.py

2.1.1 The scripts for command line: train and dev data from the same source
1) polnear_BIO: 
python3 CRF.py polnear_preprocessed_BIO_train_token_label.csv polnear_preprocessed_BIO_dev_token_label.csv CRF_out_BIO_polnear.csv
2) parc3_BIO:
python3 CRF.py parc3_preprocessed_BIO_train_token_label.csv parc3_preprocessed_BIO_dev_token_label.csv CRF_out_BIO_parc3.csv
3) merged_parc3_polnear_BIO:
python3 CRF.py merged-parc3-polnear_preprocessed_BIO_train_token_label.csv merged-parc3-polnear_preprocessed_BIO_dev_token_label.csv CRF_out_BIO_merged-parc3-polnear.csv

2.1.2 The scripts for command line: train and dev data from the different source
1) polnear_BIO: 
python3 CRF.py polnear_preprocessed_BIO_train_token_label.csv parc3_preprocessed_BIO_dev_token_label.csv CRF_out_BIO_polnear-parc3.csv
2) parc3_BIO:
python3 CRF.py parc3_preprocessed_BIO_train_token_label.csv polnear_preprocessed_BIO_dev_token_label.csv CRF_out_BIO_parc3-polnear.csv

2.2.1 The scripts for command line: train and dev data from the same source, unlabeled sentences removed
1) polnear_BIO: 
python3 CRF.py polnear_preprocessed_BIO_train_token_label_removed.csv polnear_preprocessed_BIO_dev_token_label_removed.csv CRF_out_BIO_polnear_removed.csv
2) parc3_BIO:
python3 CRF.py parc3_preprocessed_BIO_train_token_label_removed.csv parc3_preprocessed_BIO_dev_token_label_removed.csv CRF_out_BIO_parc3_removed.csv
3) merged_parc3_polnear_BIO:
python3 CRF.py merged-parc3-polnear_preprocessed_BIO_train_token_label_removed.csv merged-parc3-polnear_preprocessed_BIO_dev_token_label_removed.csv CRF_out_BIO_merged-parc3-polnear_removed.csv

2.2.2 The scripts for command line: train and dev data from the different source, unlabeled sentences removed
1) polnear_BIO: 
python3 CRF.py polnear_preprocessed_BIO_train_token_label_removed.csv parc3_preprocessed_BIO_dev_token_label_removed.csv CRF_out_BIO_polnear-parc3_removed.csv
2) parc3_BIO:
python3 CRF.py parc3_preprocessed_BIO_train_token_label_removed.csv polnear_preprocessed_BIO_dev_token_label_removed.csv CRF_out_BIO_parc3-polnear_removed.csv

3.1.1 Evaluation: train and dev data from the same source. unlabeled sentences not removed

In [22]:
# https://www.geeksforgeeks.org/python-read-csv-columns-into-list/
# to write the annotations and predictions into lists

# To get the predictions
result_list=["CRF_out_BIO_polnear.csv","CRF_out_BIO_parc3.csv","CRF_out_BIO_merged-parc3-polnear.csv"]
for i in result_list:
    filename = open(f'Result/{i}', 'r')
    file = csv.DictReader(filename, delimiter='\t')
    four_class_system_output = []
    tokens=[]
    for col in file:
        #print(col)
        four_class_system_output.append(col['pred_AR_label'])
        tokens.append(col['Word'])
    #print(four_class_system_output[0:20])
    # To get the annotations
    data_source=i.replace('.csv','').split("_")[-1]
    filename1 = open(f'../Preprocessed_data/{data_source}_preprocessed_BIO_dev_token_label.csv', 'r')
    file1 = csv.DictReader(filename1, delimiter='\t')
    four_class_human_annotation = []
    for col in file1:
        #print(col)
        four_class_human_annotation.append(col['AR_label'])
    # to check if the predicted file and the dev file are aligned.
    #print(f"{data_source} {len(four_class_human_annotation)} and {len(four_class_system_output)}")
    report = classification_report(four_class_human_annotation[:len(four_class_system_output)],four_class_system_output,digits = 2)
    print(f"Baseline evaluation {data_source} BIO with {data_source}'s train and dev files")
    print(report)
    print(sklearn.metrics.confusion_matrix(four_class_human_annotation[:len(four_class_system_output)],four_class_system_output))
    print("----------------------------------------------------------")
    print()


Baseline evaluation polnear BIO with polnear's train and dev files
              precision    recall  f1-score   support

   B-CONTENT       0.59      0.40      0.48      2193
       B-CUE       0.80      0.56      0.66      2189
    B-SOURCE       0.72      0.50      0.59      1947
   I-CONTENT       0.74      0.74      0.74     36877
       I-CUE       0.42      0.23      0.30      1808
    I-SOURCE       0.59      0.46      0.52      4070
           O       0.66      0.73      0.70     32460

    accuracy                           0.69     81544
   macro avg       0.65      0.52      0.57     81544
weighted avg       0.69      0.69      0.69     81544

[[  883     1    63   531    75    21   619]
 [    1  1221     3   443    19    27   475]
 [   24     7   971   371     5    46   523]
 [  336   122    72 27216   237   315  8579]
 [   38    44     5   554   423    55   689]
 [   10     8    76   719    13  1867  1377]
 [  199   117   150  7099   224   817 23854]]
--------------------

3.1.2 Evaluation: train and dev data from the different source. unlabeled sentences not removed

In [78]:
# https://www.geeksforgeeks.org/python-read-csv-columns-into-list/
# to write the annotations and predictions into lists

# To get the predictions
result_list=["CRF_out_BIO_polnear-parc3.csv","CRF_out_BIO_parc3-polnear.csv"]
for i in result_list:
    filename = open(f'Result/{i}', 'r')
    file = csv.DictReader(filename, delimiter='\t')
    four_class_system_output = []
    tokens=[]
    for col in file:
        #print(col)
        four_class_system_output.append(col['pred_AR_label'])
        tokens.append(col['Word'])
    #print(four_class_system_output[0:20])
    # To get the annotations
    data_source=i.replace('.csv','').split("_")[-1]
    train_data=data_source.split("-")[0]
    dev_data=data_source.split("-")[1]  
    filename1 = open(f'../Preprocessed_data/{dev_data}_preprocessed_BIO_dev_token_label.csv', 'r')
    file1 = csv.DictReader(filename1, delimiter='\t')
    four_class_human_annotation = []
    for col in file1:
        #print(col)
        four_class_human_annotation.append(col['AR_label'])
    # to check if the predicted file and the dev file are aligned.
    #print(f"{data_source} {len(four_class_human_annotation)} and {len(four_class_system_output)}")
    report = classification_report(four_class_human_annotation[:len(four_class_system_output)],four_class_system_output,digits = 2)
    print(f"Baseline evaluation {data_source} BIO with {train_data}'s train and {dev_data}'s dev files")
    print(report)
    print(sklearn.metrics.confusion_matrix(four_class_human_annotation[:len(four_class_system_output)],four_class_system_output))
    print("----------------------------------------------------------")
    print()


Baseline evaluation polnear-parc3 BIO with polnear's train and parc3's dev files
              precision    recall  f1-score   support

   B-CONTENT       0.08      0.05      0.06       627
       B-CUE       0.12      0.07      0.09       650
    B-SOURCE       0.07      0.03      0.04       579
   I-CONTENT       0.36      0.36      0.36      9817
       I-CUE       0.02      0.01      0.01       276
    I-SOURCE       0.08      0.03      0.05      1671
           O       0.64      0.71      0.68     20439

    accuracy                           0.53     34059
   macro avg       0.20      0.18      0.18     34059
weighted avg       0.50      0.53      0.52     34059

[[   31    15     4   196     5    12   364]
 [    5    46     3   203     3    16   374]
 [    8     7    19   179     1    10   355]
 [  128   115    99  3495    69   216  5695]
 [    5     4     1    97     3     1   165]
 [   22    17    16   498     8    57  1053]
 [  173   182   124  4931   105   378 14546]]
------

3.2.1 Evaluation: train and dev data from the same source. unlabeled sentences removed

In [79]:
# https://www.geeksforgeeks.org/python-read-csv-columns-into-list/
# to write the annotations and predictions into lists

# To get the predictions
result_list=["CRF_out_BIO_polnear_removed.csv","CRF_out_BIO_parc3_removed.csv"]
for i in result_list:
    filename = open(f'Result/{i}', 'r')
    file = csv.DictReader(filename, delimiter='\t')
    four_class_system_output = []
    tokens=[]
    for col in file:
        #print(col)
        four_class_system_output.append(col['pred_AR_label'])
        tokens.append(col['Word'])
    #print(four_class_system_output[0:20])
    # To get the annotations
    data_source=i.replace('.csv','').split("_")[-2]
    #train_data=data_source.split("-")[0]
    #dev_data=data_source.split("-")[1]  
    filename1 = open(f'../Preprocessed_data/{data_source}_preprocessed_BIO_dev_token_label_removed.csv', 'r')
    file1 = csv.DictReader(filename1, delimiter='\t')
    four_class_human_annotation = []
    for col in file1:
        #print(col)
        four_class_human_annotation.append(col['AR_label'])
    # to check if the predicted file and the dev file are aligned.
    #print(f"{data_source} {len(four_class_human_annotation)} and {len(four_class_system_output)}")
    report = classification_report(four_class_human_annotation[:len(four_class_system_output)],four_class_system_output,digits = 2)
    print(f"Baseline evaluation {data_source} BIO with {data_source}'s train and dev files")
    print(report)
    print(sklearn.metrics.confusion_matrix(four_class_human_annotation[:len(four_class_system_output)],four_class_system_output))
    print("----------------------------------------------------------")
    print()


Baseline evaluation polnear BIO with polnear's train and dev files
              precision    recall  f1-score   support

   B-CONTENT       0.59      0.44      0.51      2192
       B-CUE       0.82      0.57      0.67      2189
    B-SOURCE       0.73      0.50      0.59      1948
   I-CONTENT       0.75      0.85      0.80     36895
       I-CUE       0.44      0.27      0.33      1807
    I-SOURCE       0.64      0.51      0.56      4070
           O       0.60      0.54      0.57     16545

    accuracy                           0.70     65646
   macro avg       0.65      0.53      0.58     65646
weighted avg       0.69      0.70      0.69     65646

[[  970     3    59   746    85    33   296]
 [    0  1240     5   642    17    47   238]
 [   54     4   973   537    10    56   314]
 [  406   130    91 31403   320   362  4183]
 [   43    53     2   822   484    74   329]
 [   12    16    81  1147    26  2060   728]
 [  164    72   114  6470   150   595  8980]]
--------------------

3.2.2 Evaluation: train and dev data from the different source. unlabeled sentences removed

In [81]:
# https://www.geeksforgeeks.org/python-read-csv-columns-into-list/
# to write the annotations and predictions into lists

# To get the predictions
result_list=["CRF_out_BIO_polnear-parc3_removed.csv","CRF_out_BIO_parc3-polnear_removed.csv"]
for i in result_list:
    filename = open(f'Result/{i}', 'r')
    file = csv.DictReader(filename, delimiter='\t')
    four_class_system_output = []
    tokens=[]
    for col in file:
        #print(col)
        four_class_system_output.append(col['pred_AR_label'])
        tokens.append(col['Word'])
    #print(four_class_system_output[0:20])
    # To get the annotations
    data_source=i.replace('.csv','').split("_")[-2]
    train_data=data_source.split("-")[0]
    dev_data=data_source.split("-")[1]  
    filename1 = open(f'../Preprocessed_data/{dev_data}_preprocessed_BIO_dev_token_label_removed.csv', 'r')
    file1 = csv.DictReader(filename1, delimiter='\t')
    four_class_human_annotation = []
    for col in file1:
        #print(col)
        four_class_human_annotation.append(col['AR_label'])
    # to check if the predicted file and the dev file are aligned.
    #print(f"{data_source} {len(four_class_human_annotation)} and {len(four_class_system_output)}")
    report = classification_report(four_class_human_annotation[:len(four_class_system_output)],four_class_system_output,digits = 2)
    print(f"Baseline evaluation {data_source} BIO with {train_data}'s train and {dev_data}'s dev files")
    print(report)
    print(sklearn.metrics.confusion_matrix(four_class_human_annotation[:len(four_class_system_output)],four_class_system_output))
    print("----------------------------------------------------------")
    print()


Baseline evaluation polnear-parc3 BIO with polnear's train and parc3's dev files
              precision    recall  f1-score   support

   B-CONTENT       0.10      0.06      0.08       627
       B-CUE       0.13      0.07      0.09       650
    B-SOURCE       0.09      0.05      0.06       579
   I-CONTENT       0.57      0.61      0.59      9817
       I-CUE       0.04      0.03      0.03       276
    I-SOURCE       0.17      0.08      0.11      1671
           O       0.30      0.37      0.33      4498

    accuracy                           0.44     18118
   macro avg       0.20      0.18      0.19     18118
weighted avg       0.41      0.44      0.42     18118

[[  40   16   16  353    4   20  178]
 [  12   48   11  362    4   22  191]
 [  20    9   27  317   11   19  176]
 [ 200  186  151 6026   87  415 2752]
 [   7    6    6  156    7    7   87]
 [  35   29   25  881   16  133  552]
 [  83   88   62 2386   50  156 1673]]
-------------------------------------------------------